In [ ]:
"""
Purpose: 
To create the function that will create a mapping correspondence
of skeletal branches (those with no node degrees higher than 3)
and the meshes themselves




"""

In [1]:
from os import sys
sys.path.append("../../meshAfterParty/")

In [327]:
import skeleton_utils as sk
import trimesh_utils as tu
import trimesh
import numpy_utils as nu
import numpy as np
from importlib import reload
import networkx as nx
import time
import compartment_utils as cu
import networkx_utils as xu


In [3]:
saved_file = "../../Platinum_Compartment/soma_touching_mesh_data.npz"
current_mesh_data = nu.load_dict(saved_file)

In [4]:
# # visualize all of the branches and the meshes
# sk.graph_skeleton_and_mesh(other_meshes=list(current_mesh_data[0]["branch_meshes"]) + list(current_mesh_data[0]["soma_meshes"]),
#                           other_meshes_colors="random",
#                            other_skeletons = current_mesh_data[0]["branch_skeletons"],
#                           other_skeletons_colors="random")

In [7]:
skelton_cleaning_threshold = 4001
total_cleaned = []
for j,curr_skeleton_to_clean in enumerate(current_mesh_data[0]["branch_skeletons"]):
    print(f"\n---- Working on Limb {j} ----")
    start_time = time.time()
    print(f"before cleaning limb size of skeleton = {curr_skeleton_to_clean.shape}")
    distance_cleaned_skeleton = sk.clean_skeleton(
                                                curr_skeleton_to_clean,
                                                distance_func=sk.skeletal_distance,
                                                min_distance_to_junction = skelton_cleaning_threshold,
                                                return_skeleton=True,
                                                print_flag=False) 
    #make sure still connected componet
    distance_cleaned_skeleton_components = nx.number_connected_components(sk.convert_skeleton_to_graph(distance_cleaned_skeleton))
    if distance_cleaned_skeleton_components > 1:
        raise Exception(f"distance_cleaned_skeleton {j} was not a single component: it was actually {distance_cleaned_skeleton_components} components")
        
    print(f"after DISTANCE cleaning limb size of skeleton = {distance_cleaned_skeleton.shape}")
    cleaned_branch = sk.clean_skeleton_with_decompose(distance_cleaned_skeleton)
    
    cleaned_branch_components = nx.number_connected_components(sk.convert_skeleton_to_graph(cleaned_branch))
    if cleaned_branch_components > 1:
        raise Exception(f"cleaned_branch {j} was not a single component: it was actually {cleaned_branch_components} components")
    
    #do the cleanin ghtat removes loops from branches
    print(f"After DECOMPOSITION cleaning limb size of skeleton = {cleaned_branch.shape}")
    print(f"Total time = {time.time() - start_time}")
    total_cleaned.append(cleaned_branch)
    
current_mesh_data[0]["branch_skeletons_cleaned"] = total_cleaned
    


---- Working on Limb 0 ----
before cleaning limb size of skeleton = (4140, 2, 3)
Using Distance measure skeletal_distance



after DISTANCE cleaning limb size of skeleton = (4004, 2, 3)
After DECOMPOSITION cleaning limb size of skeleton = (4004, 2, 3)
Total time = 1.2711188793182373

---- Working on Limb 1 ----
before cleaning limb size of skeleton = (2975, 2, 3)
Using Distance measure skeletal_distance



after DISTANCE cleaning limb size of skeleton = (2936, 2, 3)
high-degree endpoints were connected so just using that connection
curr_branch = {2179}
After DECOMPOSITION cleaning limb size of skeleton = (2934, 2, 3)
Total time = 0.8379015922546387

---- Working on Limb 2 ----
before cleaning limb size of skeleton = (2025, 2, 3)
Using Distance measure skeletal_distance



after DISTANCE cleaning limb size of skeleton = (1981, 2, 3)
high-degree endpoints were connected so just using that connection
curr_branch = {1547}
After DECOMPOSITION cleaning limb size of skeleton = (1979, 2, 3)
Total time = 0.5441508293151855

---- Working on Limb 3 ----
before cleaning limb size of skeleton = (1840, 2, 3)
Using Distance measure skeletal_distance



after DISTANCE cleaning limb size of skeleton = (1804, 2, 3)
After DECOMPOSITION cleaning limb size of skeleton = (1804, 2, 3)
Total time = 0.5659165382385254

---- Working on Limb 4 ----
before cleaning limb size of skeleton = (2273, 2, 3)
Using Distance measure skeletal_distance


...


after DISTANCE cleaning limb size of skeleton = (1107, 2, 3)
After DECOMPOSITION cleaning limb size of skeleton = (1107, 2, 3)
Total time = 0.31572604179382324

---- Working on Limb 6 ----
before cleaning limb size of skeleton = (1063, 2, 3)
Using Distance measure skeletal_distance



after DISTANCE cleaning limb size of skeleton = (1034, 2, 3)
After DECOMPOSITION cleaning limb size of skeleton = (1034, 2, 3)
Total time = 0.30802202224731445

---- Working on Limb 7 ----
before cleaning limb size of skeleton = (1052, 2, 3)
Using Distance measure skeletal_distance



after DISTANCE cleaning limb size of skeleton = (1013, 2, 3)
After DECOMPOSITION cleaning limb size of skeleton = (1013, 2, 3)
Total time = 0.2943391799926758

---- Working on Limb 8 ----
before cleaning limb size of skeleton = (687, 2, 3)
Using Distance measure skeletal_distance



after DISTANCE cleaning limb size of skeleton = (671, 2, 3)
After DECOMPOSITION cleaning limb size of skeleton = (671, 2, 3)
Total time = 0.19977927207946777

---- Working on Limb 9 ----
before cleaning limb size of skeleton = (254, 2, 3)
Using Distance measure skeletal_distance



after DISTANCE cleaning limb size of skeleton = (241, 2, 3)
After DECOMPOSITION cleaning limb size of skeleton = (241, 2, 3)
Total time = 0.09128260612487793

---- Working on Limb 10 ----
before cleaning limb size of skeleton = (31, 2, 3)
Using Distance measure skeletal_distance
no small end nodes to get rid of so returning whole skeleton
after DISTANCE cleaning limb size of skeleton = (31, 2, 3)
only one skeleton so no stacking needed
After DECOMPOSITION cleaning limb size of skeleton = (31, 2, 3)
Total time = 0.01213693618774414


In [ ]:
# # visualize all of the branches and the meshes
# sk.graph_skeleton_and_mesh(other_meshes=list(current_mesh_data[0]["branch_meshes"]) + list(current_mesh_data[0]["soma_meshes"]),
#                           other_meshes_colors="random",
#                            other_skeletons = current_mesh_data[0]["branch_skeletons_cleaned"],
#                           other_skeletons_colors="random")

# checking that all the cleaned branches are connected components

In [8]:
for k,cl_sk in enumerate(current_mesh_data[0]["branch_skeletons"]): 
    n_components = nx.number_connected_components(sk.convert_skeleton_to_graph(cl_sk)) 
    if n_components > 1:
        raise Exception(f"Original limb {k} was not a single component: it was actually {n_components} components")
    
for k,cl_sk in enumerate(current_mesh_data[0]["branch_skeletons_cleaned"]): 
    n_components = nx.number_connected_components(sk.convert_skeleton_to_graph(cl_sk)) 
    if n_components > 1:
        raise Exception(f"Cleaned limb {k} was not a single component: it was actually {n_components} components")

# new correspondence function

In [9]:
from skeleton_utils import *

def get_skeletal_distance(main_mesh,edges,
                                 buffer=0.01,
                                bbox_ratio=1.2,
                               distance_threshold=3000,
                                print_flag=False):
    """
    Purpose: To return the histogram of distances along a mesh subtraction process
    so that we could evenutally find an adaptive distance threshold
    
    
    """

    
    main_mesh_bbox_restricted = main_mesh
    faces_bbox_inclusion = np.arange(0,len(main_mesh.faces))

    
    start_time = time.time()
    face_subtract_indices = []
    
    edge_loop_print=True
    total_distances = []
    total_distances_std = []
    for i,ex_edge in tqdm(enumerate(edges)):
        #print("\n------ New loop ------")
        #print(ex_edge)
        
        # ----------- creating edge and checking distance ----- #
        loop_start = time.time()
        
        edge_line = ex_edge[1] - ex_edge[0]
        sum_threshold = 0.001
        if np.sum(np.abs(edge_line)) < sum_threshold:
            if edge_loop_print:
                print(f"edge number {i}, {ex_edge}: has sum less than {sum_threshold} so skipping")
            continue
        
        cob_edge = change_basis_matrix(edge_line)
        
        #get the limits of the example edge itself that should be cutoff
        edge_trans = (cob_edge@ex_edge.T)
        #slice_range = np.sort((cob_edge@ex_edge.T)[2,:])
        slice_range = np.sort(edge_trans[2,:])

        # adding the buffer to the slice range
        slice_range_buffer = slice_range + np.array([-buffer,buffer])

        # generate face midpoints from the triangles
        #face_midpoints = np.mean(main_mesh_bbox_restricted.vertices[main_mesh_bbox_restricted.faces],axis=1) # Old way
        face_midpoints = main_mesh_bbox_restricted.triangles_center
        
        
        #get the face midpoints that fall within the slice (by lookig at the z component)
        fac_midpoints_trans = cob_edge@face_midpoints.T
        
        slice_mask_pre_distance = ((fac_midpoints_trans[2,:]>slice_range_buffer[0]) & 
                      (fac_midpoints_trans[2,:]<slice_range_buffer[1]))

        edge_midpoint = np.mean(edge_trans.T,axis=0)
        distance_check = np.linalg.norm((fac_midpoints_trans.T)[:,:2] - edge_midpoint[:2],axis=1) < distance_threshold
        
        slice_mask = slice_mask_pre_distance & distance_check
        
        face_list = np.arange(0,len(main_mesh_bbox_restricted.faces))[slice_mask]

        #get the submesh of valid faces within the slice
        if len(face_list)>0:
            main_mesh_sub = main_mesh_bbox_restricted.submesh([face_list],append=True)
        else:
            main_mesh_sub = []

        if type(main_mesh_sub) != type(trimesh.Trimesh()):
            if edge_loop_print:
                print(f"THERE WERE NO FACES THAT FIT THE DISTANCE ({distance_threshold}) and Z transform requirements")
                print("So just skipping this edge")
            continue


        #get all disconnected mesh pieces of the submesh and the face indices for lookup later
        sub_components,sub_components_face_indexes = tu.split(main_mesh_sub,only_watertight=False)
       
        
        
        if type(sub_components) != type(np.array([])) and type(sub_components) != list:
            #print(f"meshes = {sub_components}, with type = {type(sub_components)}")
            if type(sub_components) == type(trimesh.Trimesh()) :
                sub_components = [sub_components]
            else:
                raise Exception("The sub_components were not an array, list or trimesh")
        

        #getting the indices of the submeshes whose bounding box contain the edge 
        contains_points_results = np.array([s_comp.bounding_box.contains(ex_edge.reshape(-1,3)) for s_comp in sub_components])
        
        containing_indices = (np.arange(0,len(sub_components)))[np.sum(contains_points_results,axis=1) >= len(ex_edge)]
        try:
            if len(containing_indices) != 1: 
                if edge_loop_print:
                    print(f"--> Not exactly one containing mesh: {containing_indices}")
                if len(containing_indices) > 1:
                    sub_components_inner = sub_components[containing_indices]
                    sub_components_face_indexes_inner = sub_components_face_indexes[containing_indices]
                else:
                    sub_components_inner = sub_components
                    sub_components_face_indexes_inner = sub_components_face_indexes

                #get the center of the edge
                edge_center = np.mean(ex_edge,axis=0)
                #print(f"edge_center = {edge_center}")

                #find the distance between eacch bbox center and the edge center
                bbox_centers = [np.mean(k.bounds,axis=0) for k in sub_components_inner]
                #print(f"bbox_centers = {bbox_centers}")
                closest_bbox = np.argmin([np.linalg.norm(edge_center-b_center) for b_center in bbox_centers])
                #print(f"bbox_distance = {closest_bbox}")
                edge_skeleton_faces = faces_bbox_inclusion[face_list[sub_components_face_indexes_inner[closest_bbox]]]


            else:# when only one viable submesh piece and just using that sole index
                edge_skeleton_faces = faces_bbox_inclusion[face_list[sub_components_face_indexes[containing_indices[0]]]]
        except:
            print(f"sub_components = {sub_components}")
            print(f"containing_indices = {containing_indices}")
            print(f"sub_components_face_indexes (from the split) = {sub_components_face_indexes}")
            raise Exception("Error occured")


        if len(edge_skeleton_faces) < 0:
            print(f"****** Warning the edge index {i}: had no faces in the edge_skeleton_faces*******")
        face_subtract_indices.append(edge_skeleton_faces)
        
        
        #---- calculating the relevant distances ---- #
        
        face_midpoints = (main_mesh_bbox_restricted.triangles_center)[edge_skeleton_faces]
        #print(f"edge_skeleton_faces.shape = {edge_skeleton_faces.shape}")
#         print(f"cob_edge = {cob_edge}")
#         print(f"face_midpoints = {face_midpoints.shape}")
#         print(f"sub_components = {sub_components}")
#         print(f"containing_indices = {containing_indices}")
#         print(f"sub_components_face_indexes (from the split) = {sub_components_face_indexes}")
        #Exception("failed on fac_midpoints_trans")
        
        fac_midpoints_trans = cob_edge@face_midpoints.T
        
            
        #print(f"fac_midpoints_trans.shape = {fac_midpoints_trans.shape}")
        mesh_slice_distances = np.linalg.norm((fac_midpoints_trans.T)[:,:2] - edge_midpoint[:2],axis=1)
        #print(f"mesh_slice_distances.shape = {mesh_slice_distances.shape}")
        
        total_distances.append(np.mean(mesh_slice_distances))
        total_distances_std.append(np.std(mesh_slice_distances))
    
    
    
    
    if len(face_subtract_indices)>0:
        all_removed_faces = np.concatenate(face_subtract_indices)

        unique_removed_faces = np.array(list(set(all_removed_faces)))
        
        if len(unique_removed_faces) <= 1:
            print(f"Distance of skeleton = {sk.calculate_skeleton_distance(edges)}")
            raise Exception(f"unique_removed_faces = {unique_removed_faces}")
            
            

        #faces_to_keep = set(np.arange(0,len(main_mesh.faces))).difference(unique_removed_faces)
        new_submesh = main_mesh.submesh([unique_removed_faces],only_watertight=False,append=True)
        
        split_meshes,components_faces = tu.split(new_submesh,return_components=True)
        
        new_submesh = split_meshes[0]
        unique_removed_faces = unique_removed_faces[components_faces[0]]
        
        #need to further restric the unique_removed_faces to those of most significant piece
    
    else:
        unique_removed_faces = np.array([])
        new_submesh = trimesh.Trimesh()
 
    
    
    return total_distances,total_distances_std,new_submesh,np.array(unique_removed_faces)


def mesh_correspondence_adaptive_distance(curr_branch_skeleton,
                                          curr_branch_mesh,
                                         skeleton_segment_width = 1000):

    #making the skeletons resized to 1000 widths and then can use outlier finding
    
    new_skeleton  = sk.resize_skeleton_branch(curr_branch_skeleton,segment_width = skeleton_segment_width)

    (segment_skeletal_mean_distances,
     segment_skeletal_std_distances,
     mesh_correspondence,
     mesh_correspondence_indices) = get_skeletal_distance(
                        main_mesh = curr_branch_mesh,
                        edges = new_skeleton,
                        buffer=100,
                        bbox_ratio=1.2,
                        distance_threshold=3000,
                        print_flag=False
    )
        
    if len(mesh_correspondence_indices)== 0:
        print("empty mesh_correspondence_indices returned so returning an empty array")
        return []
    
    #now use the new submesh to calculate the new threshold
    # -- Step where I compute the new threshold and and then rerun it -- #
    if len(segment_skeletal_mean_distances) > 4:
        filtered_measurements = np.array(segment_skeletal_mean_distances[1:-1])
        filtered_measurements_std = np.array(segment_skeletal_std_distances[1:-1])
    else:
        filtered_measurements = np.array(segment_skeletal_mean_distances)
        filtered_measurements_std = np.array(segment_skeletal_std_distances)



    #filter out the other outliers: do anything higher than 150% of median should be discounted
    median_value = np.median(filtered_measurements)
    outlier_mask = filtered_measurements < median_value*1.5
    filtered_measurements = filtered_measurements[outlier_mask]
    filtered_measurements_std = filtered_measurements_std[outlier_mask]

    print(f"filtered_measurements = {filtered_measurements}")

    np.mean(filtered_measurements),np.std(filtered_measurements)

    # try the mesh subtraction again 
    buffer = 100

    total_threshold = np.max(filtered_measurements) + 2*np.max(filtered_measurements_std)
    print(f"new_threshold = {total_threshold}")
    (segment_skeletal_mean_distances_2,
     filtered_measurements_std,
     mesh_correspondence_2,
     mesh_correspondence_indices_2) = get_skeletal_distance(
                        main_mesh = mesh_correspondence,
                        edges = new_skeleton,
                        buffer=100,
                        bbox_ratio=1.2,
                        distance_threshold=total_threshold,
                        print_flag=False
    )
    
    if len(mesh_correspondence_indices_2) == 0:
        print("empty mesh_correspondence_indices_2 returned so returning an empty array")
        return []
        
        
    #want to show the changes in mesh
#     sk.graph_skeleton_and_mesh(other_meshes = [curr_branch_mesh.submesh([mesh_correspondence_indices],append=True)])
#     sk.graph_skeleton_and_mesh(other_meshes = [curr_branch_mesh.submesh([mesh_correspondence_indices[mesh_correspondence_indices_2]],append=True)])
        
    # PROBLEM NOT PASSING BACK A CONNECTED COMPONENT
        
    return mesh_correspondence_indices[mesh_correspondence_indices_2], total_threshold

# -- decomposing all of limbs into branches and finding mesh correspondence using adaptive manner -- 

In [10]:

sk = reload(sk)
tu = reload(tu)
from tqdm.notebook import tqdm
import time
import itertools

start_time = time.time()

limb_correspondence = dict()
soma_containing_idx= 0

total_branch_meshes = []
total_branch_skeletons = []

for soma_containing_idx in current_mesh_data.keys():
    for limb_idx,curr_limb_mesh in enumerate(current_mesh_data[soma_containing_idx]["branch_meshes"]):
        print(f"Working on limb #{limb_idx}")
        limb_correspondence[limb_idx] = dict()
        curr_limb_sk = current_mesh_data[soma_containing_idx]["branch_skeletons_cleaned"][limb_idx]
        curr_limb_branches_sk_uneven = sk.decompose_skeleton_to_branches(curr_limb_sk) #the line that is decomposing to branches
        
        for j,curr_branch_sk in tqdm(enumerate(curr_limb_branches_sk_uneven)):
            limb_correspondence[limb_idx][j] = dict()
            
            
            curr_branch_face_correspondence, width_from_skeleton = mesh_correspondence_adaptive_distance(curr_branch_sk,
                                          curr_limb_mesh,
                                         skeleton_segment_width = 1000)
            
            
            
            if len(curr_branch_face_correspondence) > 0:
                curr_submesh = curr_limb_mesh.submesh([list(curr_branch_face_correspondence)],append=True)
            else:
                curr_submesh = trimesh.Trimesh(vertices=np.array([]),faces=np.array([]))
                
            limb_correspondence[limb_idx][j]["branch_skeleton"] = curr_branch_sk
            limb_correspondence[limb_idx][j]["correspondence_mesh"] = curr_submesh
            limb_correspondence[limb_idx][j]["correspondence_face_idx"] = curr_branch_face_correspondence
            limb_correspondence[limb_idx][j]["width_from_skeleton"] = width_from_skeleton
            
            total_branch_meshes.append(curr_submesh)
            total_branch_skeletons.append(curr_branch_sk)
            
            
print(f"Total time = {time.time() - start_time}")

Working on limb #0


new_skeleton = (44, 2, 3)



filtered_measurements = [235.35614141 295.50582798 282.58422455 208.45049582 111.84423036
 240.15326795 195.22957467 179.6046358  215.70337332 183.76334931
 222.86183322 291.68475329 375.64066972 232.38761407 253.5265144
 401.78139825 280.00784496 219.4453072  342.14479643 430.29074126
 214.73273392 273.92422933 401.37785497 302.19686064 314.01544074
 254.26722848 216.49610007 174.61311501 360.33120724 373.8862993
 323.38882963]
new_threshold = 1018.9984891875927



new_skeleton = (12, 2, 3)


--> Not exactly one containing mesh: []

filtered_measurements = [ 68.91491071  98.98806728  99.46925241  97.87159894 119.97255252
  80.50298124  81.73595318 118.83832399]
new_threshold = 242.15564086496752


...


filtered_measurements = [110.46231186 124.35507099 105.77243209  73.15927553  89.09908199
 167.76382586 122.91891077 153.3331216 ]
new_threshold = 278.3739552321925




Working on limb #10


new_skeleton = (15, 2, 3)



filtered_measurements = [640.8730923  693.94094553 660.83003566 581.18392316 584.0527867
 521.07311147 491.66561022 478.88269115 443.34973484 405.31585196
 404.49025627 347.55558462 602.12282632]
new_threshold = 1452.8343419324337




Total time = 164.0838279724121


# Checking that there are no empty meshes 

In [11]:
total_decomposed_branches = []
for soma_containing_idx in current_mesh_data.keys():
    for limb_idx,curr_limb_mesh in enumerate(current_mesh_data[soma_containing_idx]["branch_meshes"]):
        print(f"Working on limb #{limb_idx}")
        curr_limb_sk = current_mesh_data[soma_containing_idx]["branch_skeletons_cleaned"][limb_idx]
        curr_limb_branches_sk = sk.decompose_skeleton_to_branches(curr_limb_sk) #the line that is decomposing to branches
        total_decomposed_branches.append(curr_limb_branches_sk)

Working on limb #0
Working on limb #1
Working on limb #2
Working on limb #3
Working on limb #4
Working on limb #5
Working on limb #6
Working on limb #7
Working on limb #8
Working on limb #9
Working on limb #10


In [12]:
len_decomposed = [len(k) for k in total_decomposed_branches]
print(len_decomposed)
print((np.sum(len_decomposed),len(total_branch_meshes)))

[75, 52, 48, 37, 34, 13, 17, 22, 11, 5, 1]
(315, 315)


In [13]:
empty_meshes = [k for k in total_branch_meshes if len(k.faces) == 0]
empty_meshes

[]

# -- Visulaizing all of branches and the mesh correspondence from adaptive version -- 

In [ ]:
sk = reload(sk)
import matplotlib_utils as mu
mu = reload(mu)
sk.graph_skeleton_and_mesh(other_meshes=total_branch_meshes,
                          other_meshes_colors="random",
                           other_skeletons=total_branch_skeletons,
                           other_skeletons_colors="random"
                          )


# The algorithm that will finish off the face correspondence

In [142]:
no_missing_labels = list(limb_correspondence[limb_idx].keys())

In [143]:
limb_idx = 0

curr_limb_mesh = current_mesh_data[soma_containing_idx]["branch_meshes"][limb_idx]

#set up the face dictionary
face_lookup = dict([(j,[]) for j in range(0,len(curr_limb_mesh.faces))])

for j,branch_piece in limb_correspondence[limb_idx].items():
    curr_faces_corresponded = branch_piece["correspondence_face_idx"]
    
    for c in curr_faces_corresponded:
        face_lookup[c].append(j)

#get all of the faces that don't have any faces corresponding
empty_indices = np.array([k for k,v in face_lookup.items() if len(v) == 0])

#get all of the faces that don't have any faces corresponding
conflict_indices = np.array([k for k,v in face_lookup.items() if len(v) >= 2])

print(f"empty_indices % = {len(empty_indices)/len(face_lookup.keys())}\n conflict_indices % = {len(conflict_indices)/len(face_lookup.keys())}")

empty_indices % = 0.07729703609292249
 conflict_indices % = 0.050641655558906803


In [144]:
original_labels = set(list(itertools.chain.from_iterable(list(face_lookup.values()))))
max(original_labels),len(original_labels)

(74, 75)

In [145]:
face_coloring = np.array([-1 if len(v) != 1 else v[0] for v in face_lookup.values()])
face_coloring

"""
face_lookup_resolved =face_lookup.copy()
# The old way of doing it using a dictionary
empty_faces = []
for k,v in face_lookup_resolved.items():
    if len(v) != 1:
        face_lookup_resolved[k] = -1
        empty_faces.append(k)
    else:
        
        face_lookup_resolved[k] = v[0]
        
#checking that they are the same
for k,v in face_lookup_resolved.items():
    if v != face_coloring[k]:
        raise Exception()"""

'\nface_lookup_resolved =face_lookup.copy()\n# The old way of doing it using a dictionary\nempty_faces = []\nfor k,v in face_lookup_resolved.items():\n    if len(v) != 1:\n        face_lookup_resolved[k] = -1\n        empty_faces.append(k)\n    else:\n        \n        face_lookup_resolved[k] = v[0]\n        \n#checking that they are the same\nfor k,v in face_lookup_resolved.items():\n    if v != face_coloring[k]:\n        raise Exception()'

In [146]:
current_indices = np.where(face_coloring==check_index)[0]
print(f"check_index = {check_index}")
sk.graph_skeleton_and_mesh(
                            other_meshes=[curr_limb_mesh.submesh([current_indices],append=True)],
                          other_meshes_colors="black",
                           other_skeletons=[divided_branches[check_index]],
                           other_skeletons_colors="black")

check_index = 48


# -- Need to only take the biggest piece of the non-conflicted mesh and resolve those that were eliminated--

In [147]:
"""
Pseudocode: 
1) For each label get the submesh
2) split the submesh into only largest piece 
3) find what faces are not included
4) get al

"""

cu = reload(cu)
face_coloring = cu.filter_face_coloring_to_connected_components(curr_limb_mesh,face_coloring)

leftover_labels = np.unique(face_coloring)
missing_labels = set(np.setdiff1d(no_missing_labels, leftover_labels))

for curr_label in missing_labels:
    labels_idx = [k for k,v in face_lookup.items() if curr_label in v]
    face_coloring[labels_idx] = curr_label

#filter the faces again: 
face_coloring = cu.filter_face_coloring_to_connected_components(curr_limb_mesh,face_coloring)

leftover_labels = np.unique(face_coloring)
missing_labels = set(np.setdiff1d(no_missing_labels, leftover_labels))

if len(missing_labels)>0:
    raise Exception("missing labels was not resolved")

In [148]:
# now just divide the groups into seperate components
empty_faces = np.where(face_coloring==-1)[0]
mesh_graph = nx.from_edgelist(curr_limb_mesh.face_adjacency)
empty_submesh = mesh_graph.subgraph(empty_faces)
len(empty_submesh.nodes())
empty_connected_components = list(nx.connected_components(empty_submesh))
len(empty_connected_components)

793

# Functions that will fill in the rest of the mesh correspondence

In [150]:
cu = reload(cu)

face_coloring_copy = face_coloring.copy()
from tqdm.notebook import tqdm
for comp in tqdm(empty_connected_components):
    face_lookup_resolved_test = cu.waterfill_labeling(
                    #total_mesh_correspondence=face_lookup_resolved_test,
                    total_mesh_correspondence=face_coloring_copy,
                     submesh_indices=list(comp),
                     total_mesh=None,
                    total_mesh_graph=mesh_graph,
                     propagation_type="random",
                    max_iterations = 1000,
                    max_submesh_threshold = 5000
                    )

All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
breaking out of loop because zero unmarked faces left after 1 iterations
All surrounding labels are the same so autofilling the remainder of unlabeled

All surrounding labels are the same so autofilling the remainder of unlabeled labels
breaking out of loop because zero unmarked faces left after 1 iterations
All surrounding labels are the same so autofilling the remainder of unlabeled labels
breaking out of loop because zero unmarked faces left after 5 iterations
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All 

breaking out of loop because zero unmarked faces left after 18 iterations
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabele

All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder of unlabeled labels
All surrounding labels are the same so autofilling the remainder 

In [151]:
empty_faces = np.where(face_coloring_copy==-1)[0]
if len(empty_faces) > 0:
    raise Exception(f"empty faces were greater than 0 after waterfilling at: {empty_faces}")


# Visualizing the Results from the Mesh correspondence

In [345]:
tu = reload(tu)
divided_submeshes,divided_submeshes_idx = tu.split_mesh_into_face_groups(curr_limb_mesh,face_coloring_copy)
len(divided_submeshes)

75

In [158]:
divided_branches = [limb_correspondence[limb_idx][k]["branch_skeleton"] for k in limb_correspondence[limb_idx]]
len(divided_branches)

75

In [159]:
overhangs = [13,22,30,33]
errors = [23]
not_tubes = [35]
too_much = [74]
way_out_of_mesh = [37,40,44,57,58,66,68,70]

In [277]:
check_index = 40
limb_correspondence[limb_idx][check_index]["width_from_skeleton"]

3636.9661269371118

In [282]:
# Visualizing one mesh that has been completely labeled
print(f"check_index = {check_index}")
print(f"divided_submeshes[check_index] = {len(divided_submeshes[check_index].faces)}")
sk.graph_skeleton_and_mesh(
                            other_meshes=[divided_submeshes[check_index]],
                          other_meshes_colors="black",
                           other_skeletons=[divided_branches[check_index]],
                           other_skeletons_colors="black")

check_index = 40
divided_submeshes[check_index] = 115


# How to make our results into a networkx version of this data

In [297]:
sk.graph_skeleton_and_mesh(other_skeletons=[ex_branch])

In [309]:

import networkx_utils as xu
xu = reload(xu)
ex_branch = divided_branches[0]
ex_graph = sk.convert_skeleton_to_graph(ex_branch)
xu.get_nodes_of_degree_k(ex_graph,1)

[0, 103]

In [311]:
#check that they all only have two nodes with a degree of 1
not_equal_to_2 = []
for j,db in enumerate(divided_branches):
    db_graph = sk.convert_skeleton_to_graph(db)
    degree_one_nodes = xu.get_nodes_of_degree_k(db_graph,1)
    if len(degree_one_nodes) != 2:
        not_equal_to_2.append(j)
        
not_equal_to_2


[]

# Creating the networkx structure

In [338]:


nu = reload(nu)
#get the endpoints of the branch
endpoint_coordinates = []
for db in divided_branches:
    db_graph = sk.convert_skeleton_to_graph(db)
    endpoint_coordinates.append(xu.get_node_attributes(db_graph,node_list=xu.get_nodes_of_degree_k(db_graph,1)))
    
endpoint_coordinates = np.array(endpoint_coordinates)
endpoint_coordinates = endpoint_coordinates.reshape(-1,3)
print(f"endpoint_coordinates.shape = {endpoint_coordinates.shape}")


# dist_matrix = nu.get_coordinate_distance_matrix(endpoint_coordinates)
# np.sum(dist_matrix[0,:] == 0)

#do the distance matrix to find 
matching_vertices = nu.get_matching_vertices(endpoint_coordinates)

#creating the actual edges 
graph_edges = np.floor(matching_vertices/2).astype("int")
limb_graph = nx.Graph(nx.from_edgelist(graph_edges))
nx.draw(limb_graph,with_labels=True)

endpoint_coordinates.shape = (150, 3)


array([[  1,   3],
       [  1,   4],
       [  3,   1],
       [  3,   4],
       [  4,   1],
       [  4,   3],
       [  5,   6],
       [  5,   8],
       [  6,   5],
       [  6,   8],
       [  7,  10],
       [  7,  12],
       [  8,   5],
       [  8,   6],
       [ 10,   7],
       [ 10,  12],
       [ 11,  15],
       [ 11,  16],
       [ 12,   7],
       [ 12,  10],
       [ 15,  11],
       [ 15,  16],
       [ 16,  11],
       [ 16,  15],
       [ 17,  18],
       [ 17,  20],
       [ 18,  17],
       [ 18,  20],
       [ 19,  23],
       [ 19,  24],
       [ 20,  17],
       [ 20,  18],
       [ 23,  19],
       [ 23,  24],
       [ 24,  19],
       [ 24,  23],
       [ 25,  27],
       [ 25,  30],
       [ 27,  25],
       [ 27,  30],
       [ 29,  37],
       [ 29,  38],
       [ 30,  25],
       [ 30,  27],
       [ 31,  33],
       [ 31,  36],
       [ 33,  31],
       [ 33,  36],
       [ 35,  45],
       [ 35,  48],
       [ 36,  31],
       [ 36,  33],
       [ 37,

# -- Adding the skeletons and meshes to the structure

In [349]:
xu = reload(xu)
dict_for_attributes = dict()
for k in limb_correspondence[limb_idx]:
    dict_for_attributes[k] = {
        "skeleton": limb_correspondence[limb_idx][k]["branch_skeleton"],
        "mesh_idx":divided_submeshes_idx[k],
        "mesh": divided_submeshes[k],
        "width":limb_correspondence[limb_idx][k]["width_from_skeleton"]
    }

xu.set_node_attributes_dict(limb_graph,dict_for_attributes)
xu.get_node_attributes(limb_graph,"skeleton")


75

# -- Filtering the wierd branches away -- #

In [351]:
# check the parts that have that ratio
width_to_face_ratio = 500/100

wierd_meshes = []
for k in limb_correspondence[limb_idx]:
    curr_width = limb_correspondence[limb_idx][k]["width_from_skeleton"]
    curr_face_count = len(divided_submeshes[k].faces)
    if curr_width/curr_face_count > width_to_face_ratio:
        wierd_meshes.append(k)
    
wierd_meshes #could then eliminate them if endpoints

limb_graph_degree_1 = get_nodes_of_degree_k(limb_graph,1)
print(f"wierd_meshes before = {wierd_meshes}")
np.setdiff1d(wierd_meshes,limb_graph_degree_1)

{0: 2, 1: 2, 2: 4, 3: 4, 4: 2, 5: 4, 6: 2, 7: 2, 8: 4, 9: 4, 10: 2, 11: 2, 12: 4, 13: 2, 15: 4, 14: 2, 18: 4, 19: 4, 16: 2, 17: 2, 22: 2, 24: 4, 27: 2, 28: 4, 20: 2, 29: 4, 30: 2, 21: 2, 23: 2, 25: 4, 50: 4, 51: 4, 34: 4, 74: 4, 26: 2, 37: 4, 38: 2, 46: 2, 47: 4, 36: 4, 41: 4, 31: 2, 33: 2, 32: 2, 35: 4, 39: 2, 59: 4, 61: 4, 40: 2, 42: 4, 43: 4, 44: 2, 48: 4, 49: 2, 45: 2, 54: 4, 62: 4, 53: 4, 56: 4, 52: 4, 57: 2, 55: 4, 60: 2, 72: 2, 73: 2, 58: 2, 63: 4, 65: 4, 64: 2, 68: 2, 69: 4, 66: 2, 67: 2, 70: 2, 71: 2}
1
wierd_meshes before = [23, 35, 37, 40, 44]


array([23, 35, 37, 40, 44])

In [ ]:
#delete the nodes you don't like 

# -- Now have to make this go for the whole neuron -- #

In [ ]:
print(check_index)
sk.graph_skeleton_and_mesh(
                            other_meshes=[limb_correspondence[limb_idx][check_index]["correspondence_mesh"]],
                          other_meshes_colors="black",
                           other_skeletons=[divided_branches[:5]],
                           other_skeletons_colors="black")

# Changes to be added later

[23, 35, 37, 40, 44]

# checking what it originally was before the waterfilling

In [273]:
print(check_index)
sk.graph_skeleton_and_mesh(
                            other_meshes=[limb_correspondence[limb_idx][check_index]["correspondence_mesh"]],
                          other_meshes_colors="black",
                           other_skeletons=[divided_branches[check_index]],
                           other_skeletons_colors="black")

58


In [274]:
#visualize in whole neuron part
sk.graph_skeleton_and_mesh(main_mesh_faces=curr_limb_mesh.faces,
                          main_mesh_verts=curr_limb_mesh.vertices,
                           edge_coordinates=current_mesh_data[soma_containing_idx]["branch_skeletons_cleaned"][limb_idx],
                           other_meshes=[limb_correspondence[limb_idx][check_index]["correspondence_mesh"]],
                           other_meshes_colors="red",
                           other_skeletons = [divided_branches[check_index]],
                           other_skeletons_colors="red"
                          )

# using the correspondence indexes

In [141]:
print(check_index)
sk.graph_skeleton_and_mesh(
                            other_meshes=[curr_limb_mesh.submesh([limb_correspondence[limb_idx][check_index]["correspondence_face_idx"]],append=True)],
                          other_meshes_colors="black",
                           other_skeletons=[divided_branches[check_index]],
                           other_skeletons_colors="black")

48


# preparing larger all limb processing

In [352]:
soma_containing_idx = 0
import time
for limb_idx in limb_correspondence.keys():
    mesh_start_time = time.time()
    #clear out the mesh correspondence if already in limb_correspondecne
    for k in limb_correspondence[limb_idx].keys():
        if "branch_mesh" in limb_correspondence[limb_idx][k]:
            del limb_correspondence[limb_idx][k]["branch_mesh"]
    
    #geting the current limb mesh
    print(f"Working on limb_correspondence for #{limb_idx}")
    no_missing_labels = list(limb_correspondence[limb_idx].keys())
    curr_limb_mesh = current_mesh_data[soma_containing_idx]["branch_meshes"][limb_idx]

    #set up the face dictionary
    face_lookup = dict([(j,[]) for j in range(0,len(curr_limb_mesh.faces))])

    for j,branch_piece in limb_correspondence[limb_idx].items():
        curr_faces_corresponded = branch_piece["correspondence_face_idx"]

        for c in curr_faces_corresponded:
            face_lookup[c].append(j)

    #get all of the faces that don't have any faces corresponding
    empty_indices = np.array([k for k,v in face_lookup.items() if len(v) == 0])

    #get all of the faces that don't have any faces corresponding
    conflict_indices = np.array([k for k,v in face_lookup.items() if len(v) >= 2])

    print(f"empty_indices % = {len(empty_indices)/len(face_lookup.keys())}\n conflict_indices % = {len(conflict_indices)/len(face_lookup.keys())}")
    
    original_labels = set(list(itertools.chain.from_iterable(list(face_lookup.values()))))
    print(f"max(original_labels),len(original_labels) = {(max(original_labels),len(original_labels))}")
    
    #doing the face coloring
    face_coloring = np.array([-1 if len(v) != 1 else v[0] for v in face_lookup.values()])
    
    # -- Need to only take the biggest piece of the non-conflicted mesh and resolve those that were eliminated--
    
    face_coloring = cu.filter_face_coloring_to_connected_components(curr_limb_mesh,face_coloring)

    leftover_labels = np.unique(face_coloring)
    missing_labels = set(np.setdiff1d(no_missing_labels.copy(), leftover_labels))
    
    # ----fixing if there were any missing labels
    
    face_coloring = cu.filter_face_coloring_to_connected_components(curr_limb_mesh,face_coloring)

    leftover_labels = np.unique(face_coloring)
    missing_labels = set(np.setdiff1d(no_missing_labels, leftover_labels))

    for curr_label in missing_labels:
        labels_idx = [k for k,v in face_lookup.items() if curr_label in v]
        face_coloring[labels_idx] = curr_label

    #filter the faces again: 
    face_coloring = cu.filter_face_coloring_to_connected_components(curr_limb_mesh,face_coloring)

    leftover_labels = np.unique(face_coloring)
    missing_labels = set(np.setdiff1d(no_missing_labels.copy(), leftover_labels))

    if len(missing_labels)>0:
        raise Exception("missing labels was not resolved")
        
    # -----now just divide the groups into seperate components
    empty_faces = np.where(face_coloring==-1)[0]
    mesh_graph = nx.from_edgelist(curr_limb_mesh.face_adjacency)
    empty_submesh = mesh_graph.subgraph(empty_faces)
    empty_connected_components = list(nx.connected_components(empty_submesh))
    
    # ---- Functions that will fill in the rest of the mesh correspondence ---- #
    cu = reload(cu)

    face_coloring_copy = face_coloring.copy()
    from tqdm.notebook import tqdm
    for comp in tqdm(empty_connected_components):
        face_lookup_resolved_test = cu.waterfill_labeling(
                        #total_mesh_correspondence=face_lookup_resolved_test,
                        total_mesh_correspondence=face_coloring_copy,
                         submesh_indices=list(comp),
                         total_mesh=None,
                        total_mesh_graph=mesh_graph,
                         propagation_type="random",
                        max_iterations = 1000,
                        max_submesh_threshold = 5000
                        )
        
    # -- wheck that the face coloring did not have any empty faces --
    empty_faces = np.where(face_coloring_copy==-1)[0]
    if len(empty_faces) > 0:
        raise Exception(f"empty faces were greater than 0 after waterfilling at: {empty_faces}")
        
    # -- splitting the mesh pieces into individual pieces
    divided_submeshes,divided_submeshes_idx = tu.split_mesh_into_face_groups(curr_limb_mesh,face_coloring_copy)

    #save off the new data as branch mesh
    for k in limb_correspondence[limb_idx].keys():
        limb_correspondence[limb_idx][k]["branch_mesh"] = divided_submeshes[k]
    
    print(f"Total time for limb mesh processing = {time.time() - mesh_start_time}")
    

SyntaxError: invalid syntax (<ipython-input-352-8a5d5ebc4a80>, line 3)